In [71]:
from src.linear_causality import LinearLKInformationFlow
import numpy as np
from tqdm import tqdm
# from causality_subspace import causality_subspace

dt=1
pn=1
T1=[];T2=[]
# pca = PCA(n_components=5)  # 假设我们想将数据降到2维  
Nt=1000000
np.random.seed = 5
# for i in tqdm(range(100)):
X1=np.random.randn(5,Nt)*0.2
X2=np.random.randn(5,Nt)*0.2

A1=np.array([[0.9,0.6,0,0.6,0],[0,0.8,0.6,0,0.6],[0,0,0.9,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T
A2=np.array([[0.2,0.6,0,0.6,0],[0,0.2,0.6,0,0.6],[0,0,0.2,0,0],[0,0,0,0.2,0],[0,0,0,0,0.2]]).T

B1=-np.array([[0,0,0,0,0],[0,0.,0,0,0],[0,0,0,0.9,0.],[0,0,0,0.,0],[0,0,0,0,0]])
B2=-np.array([[0,0,0,0,0],[0,0.0,0,0,0],[0,0,0,0,0],[0,0,0,0.4,0],[0,0,0,0,0]])
for it in range(1,Nt):
    X1[:,it]=A1@X1[:,it-1]+X1[:,it]+1*B1@X2[:,it-1] 
    X2[:,it]=A2@X2[:,it-1]+X2[:,it]+1*B2@X1[:,it-1] 
    
XX=np.zeros([10,Nt])
XX[:5]=X1
XX[5:]=X2

In [78]:
segments = [(2,3),(8,9)]
segments = [(0, 5),(5,10)]

In [112]:
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(XX[:,-15000:].T, lag_list=[1], segments=segments)

In [113]:
result_dict = lkif.get_dict()

In [114]:
np.abs(result_dict["information_flow"]) > result_dict["information_flow_std_origin"]*2.56

array([[ True,  True],
       [ True,  True]])

In [115]:
np.abs(result_dict["information_flow"]) > result_dict["statistics"]["p99_critical_value"]

array([[ True,  True],
       [ True,  True]])

In [111]:
np.abs(result_dict["information_flow"])

array([[2.00481158e+00, 3.38060437e-03],
       [2.47244279e-01, 4.00879888e+00]])

In [105]:
result_dict["information_flow_std_origin"]

array([[0.01538825, 0.0014133 ],
       [0.00431541, 0.01945412]])

In [106]:
result_dict["information_flow_std"]

array([[0.01538825, 0.00157665],
       [0.00631235, 0.01945412]])

---

In [116]:
## load data/X.dat and Y.dat
import pandas as pd
data_X = pd.read_csv('data/X.dat', header=None)
data_Y = pd.read_csv('data/Y.dat', header=None)

In [117]:
data_X = data_X.to_numpy()
data_Y = data_Y.to_numpy()

In [118]:
data = np.concatenate((data_X, data_Y), axis=1)

In [119]:
segments = [(0,1),(1,2)]

In [128]:
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(data, lag_list=[1], segments=segments)

In [129]:
result_dict = lkif.get_dict()

In [143]:
print('## 是否存在因果关系 (False 代表无信息流)\n')
print("### 使用原本的方差检验方法")
print("```")
print(np.abs(result_dict["information_flow"]) > result_dict["information_flow_std_origin"] * 1.65)
print("```")

print("### 使用新的方差检验方法")
print("```")
print(np.abs(result_dict["information_flow"]) > result_dict["information_flow_std"] * 1.65)
print("```")


## 是否存在因果关系 (False 代表无信息流)

### 使用原本的方差检验方法
```
[[ True False]
 [ True  True]]
```
### 使用新的方差检验方法
```
[[ True False]
 [False  True]]
```


---

In [1]:
## bootstrap
from src.linear_causality import LinearLKInformationFlow
import numpy as np
from tqdm import tqdm
# from causality_subspace import causality_subspace

dt=1
pn=1
T1=[];T2=[]
# pca = PCA(n_components=5)  # 假设我们想将数据降到2维  
Nt=100000
np.random.seed = 5
# for i in tqdm(range(100)):
X1=np.random.randn(3,Nt)*0.2
X2=np.random.randn(3,Nt)*0.2

A1=np.array([[0.9,0.6,0,],[0,0.8,0.6,],[0,0,0.9,]]).T
A2=np.array([[0.2,0.6,0,],[0,0.2,0.6,],[0,0,0.2]]).T

B1=-np.array([[0,0,0],[0,0.,0],[0,0,0]])
B2=-np.array([[0,0,0],[0,0.,0],[0,0,0]])
for it in range(1,Nt):
    X1[:,it]=A1@X1[:,it-1]+X1[:,it]+1*B1@X2[:,it-1] 
    X2[:,it]=A2@X2[:,it-1]+X2[:,it]+1*B2@X1[:,it-1] 
    
XX=np.zeros([6,Nt])
XX[:3]=X1
XX[3:]=X2

In [6]:
# bootstrap
import numpy as np
from sklearn.utils import resample
from tqdm import tqdm


In [7]:
def bootstrap_resample(data, n_resamples=1000):
    resamples = []
    for _ in range(n_resamples):
        resample = data[np.random.choice(data.shape[0], size=data.shape[0], replace=True), :]
        resamples.append(resample)
    return np.array(resamples)

In [8]:
a = bootstrap_resample(XX.T, n_resamples=1000)

KeyboardInterrupt: 

In [ ]:
segments = [(0,1),(1,2),(2,3),(3,4),(4,5),(5,6)]
lkif = LinearLKInformationFlow(np,1)
lkif.causality_estimate(XX[:,-15000:].T, lag_list=[1], segments=segments)
result_dict = lkif.get_dict()

In [18]:
print(np.abs(result_dict["information_flow"]) > result_dict["information_flow_std_origin"] * 1.65)

[[ True False False False False False]
 [ True  True False False False  True]
 [False  True  True False False False]
 [False False False  True False False]
 [False False False  True  True False]
 [ True False False False  True  True]]


In [16]:
np.concatenate([np.concatenate([A1,B1],axis=1),np.concatenate([B2,A2],axis=1)],axis=0)

array([[ 0.9,  0. ,  0. , -0. , -0. , -0. ],
       [ 0.6,  0.8,  0. , -0. , -0. , -0. ],
       [ 0. ,  0.6,  0.9, -0. , -0. , -0. ],
       [-0. , -0. , -0. ,  0.2,  0. ,  0. ],
       [-0. , -0. , -0. ,  0.6,  0.2,  0. ],
       [-0. , -0. , -0. ,  0. ,  0.6,  0.2]])